In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)

def generate_tornado_samples(label, n, center):
    return pd.DataFrame({
        "storm_relative_helicity": np.random.normal(center["srh"], 30, n),
        "CAPE":                     np.random.normal(center["cape"], 400, n),
        "lifted_condensation_level": np.random.normal(center["lcl"], 150, n),
        "bulk_wind_shear":         np.random.normal(center["shear"], 3.0, n),
        "significant_tornado_param": np.random.normal(center["stp"], 0.4, n),
        "tornado_binary":          label
    })

midwest_supercell = generate_tornado_samples(1, 1200, {
    "srh": 320, "cape": 2800, "lcl": 850, "shear": 20, "stp": 2.5
})

southern_plains_high_lcl = generate_tornado_samples(0, 900, {
    "srh": 180, "cape": 2800, "lcl": 1700, "shear": 18, "stp": 0.9
})

cool_saturated_dud = generate_tornado_samples(0, 700, {
    "srh": 220, "cape": 900, "lcl": 700, "shear": 10, "stp": 0.5
})

classic_tornado_day = generate_tornado_samples(1, 1000, {
    "srh": 350, "cape": 3200, "lcl": 800, "shear": 22, "stp": 3.0
})

marginal_case = generate_tornado_samples(1, 600, {
    "srh": 250, "cape": 2300, "lcl": 1000, "shear": 14, "stp": 1.5
})

# --- Final Assembly ---
data = pd.concat([
    midwest_supercell,
    southern_plains_high_lcl,
    cool_saturated_dud,
    classic_tornado_day,
    marginal_case
])
data = data.clip(lower=0).sample(frac=1, random_state=42).reset_index(drop=True)
data.to_csv("dataset/tornado_data.csv", index=False)

print("TornadoNet dataset generated:", data.shape)

TornadoNet dataset generated: (4400, 6)


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.saving import register_keras_serializable
from sklearn.model_selection import train_test_split

data = pd.read_csv("dataset/tornado_data.csv")
X = data.drop("tornado_binary", axis=1).astype("float32")
y = data["tornado_binary"].astype("float32")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)

@register_keras_serializable()
class CAPEAmplifier(tf.keras.layers.Layer):
    def __init__(self, threshold=2000, scale=0.001, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.scale = scale

    def call(self, inputs):
        cape = inputs[:, 1]
        boost = tf.sigmoid((cape - self.threshold) * self.scale)
        mod = 1.0 + 0.3 * boost
        return tf.expand_dims(mod, axis=-1)

@register_keras_serializable()
class LCLSuppressor(tf.keras.layers.Layer):
    def __init__(self, threshold=1400, scale=0.002, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.scale = scale

    def call(self, inputs):
        lcl = inputs[:, 2]
        suppression = tf.sigmoid((lcl - self.threshold) * self.scale)
        return tf.expand_dims(1.0 - 0.25 * suppression, axis=-1)

@register_keras_serializable()
class STPActivator(tf.keras.layers.Layer):
    def __init__(self, threshold=1.5, scale=1.0, **kwargs):
        super().__init__(**kwargs)
        self.threshold = threshold
        self.scale = scale

    def call(self, inputs):
        stp = inputs[:, 4]
        activation = tf.sigmoid((stp - self.threshold) * self.scale)
        return tf.expand_dims(1.0 + 0.2 * activation, axis=-1)

@register_keras_serializable()
class ModulationMixer(tf.keras.layers.Layer):
    def call(self, inputs):
        cape_mod, lcl_mod, stp_mod = inputs
        combined = cape_mod * lcl_mod * stp_mod
        return 1.0 + 0.3 * tf.tanh(combined - 1.0)

input_layer = layers.Input(shape=(5,), name="tornado_inputs")

x = layers.BatchNormalization()(input_layer)
x1 = layers.Dense(64, activation="relu")(x)
x2 = layers.Dense(32, activation="relu")(x1)
x3 = layers.Dense(16, activation="relu")(x2)
base_logits = layers.Dense(1)(x3)

cape_mod = CAPEAmplifier()(input_layer)
lcl_mod  = LCLSuppressor()(input_layer)
stp_mod  = STPActivator()(input_layer)
mod_signal = ModulationMixer()([cape_mod, lcl_mod, stp_mod])

combined_logits = layers.Add()([base_logits, mod_signal])
final_output = layers.Activation("sigmoid")(combined_logits)

model = models.Model(inputs=input_layer, outputs=final_output)
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0.05),
    metrics=["accuracy"]
)

early_stop = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=15, batch_size=16, callbacks=[early_stop])
model.save('models/TornadoNet.h5')
import tf2onnx
loss, acc = model.evaluate(X_test, y_test)
print(f"TornadoNet Accuracy: {acc:.4f}")
model_proto, _ = tf2onnx.convert.from_keras(model)
with open(f"models/ONNX/TornadoNet.onnx", "wb") as f:
    f.write(model_proto.SerializeToString())



Epoch 1/15
154/154 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8288 - loss: 0.4123 - val_accuracy: 0.9529 - val_loss: 0.2193
Epoch 2/15
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9632 - loss: 0.1959 - val_accuracy: 0.9935 - val_loss: 0.1452
Epoch 3/15
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9649 - loss: 0.1871 - val_accuracy: 0.9935 - val_loss: 0.1412
Epoch 4/15
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9732 - loss: 0.1746 - val_accuracy: 0.9903 - val_loss: 0.1442
Epoch 5/15
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9771 - loss: 0.1722 - val_accuracy: 0.9919 - val_loss: 0.1355
Epoch 6/15
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9747 - loss: 0.1730 - val_accuracy: 0.9935 - val_loss: 0.1351
Epoch 7/15
154/154 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.9749 - loss: 0.1662 - val_accuracy: 0.9919 - val_loss: 0.1368
Epoch 8/15
154/154 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9754 - loss: 0.1665 - val_accuracy: 0

rewriter <function rewrite_constant_fold at 0x0000018AD6E820C0>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.


In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

scenarios = [
    {
        "label": "🌩️ Classic Supercell Day",
        "features": [340, 3100, 820, 22, 2.9],
        "expected": "Tornado"
    },
    {
        "label": "🌤️ High LCL, Dry Thunderstorm",
        "features": [190, 2700, 1750, 16, 0.8],
        "expected": "No Tornado"
    },
    {
        "label": "🌪️ Marginal Shear, High Instability",
        "features": [240, 3000, 950, 12, 1.6],
        "expected": "Possibly Tornado"
    },
    {
        "label": "🌫️ Cool Humid Blip",
        "features": [210, 950, 680, 8, 0.6],
        "expected": "No Tornado"
    },
    {
        "label": "⚡ STP-Peaked Composite Storm",
        "features": [320, 2800, 870, 20, 3.1],
        "expected": "Tornado"
    },
    {
        "label": "🌀 Low Vorticity High CAPE",
        "features": [270, 3200, 1000, 10, 0.9],
        "expected": "Edge Case"
    }
]

# --- Evaluate ---
print("\n🌪️ TornadoNet Scenario Evaluation:\n")
for case in scenarios:
    x = np.array(case["features"], dtype="float32").reshape(1, -1)
    pred = model(x).numpy()[0][0]
    verdict = (
        "Tornado" if pred > 0.55 else
        "Possibly Tornado" if 0.4 < pred <= 0.55 else
        "No Tornado"
    )

    print(f"{case['label']}")
    print(f"  ➤ Features     : {case['features']}")
    print(f"  ➤ Predicted    : {verdict} (confidence: {pred:.2f})")
    print(f"  ➤ Expected     : {case['expected']}\n")


🌪️ TornadoNet Scenario Evaluation:

🌩️ Classic Supercell Day
  ➤ Features     : [340, 3100, 820, 22, 2.9]
  ➤ Predicted    : Tornado (confidence: 0.97)
  ➤ Expected     : Tornado

🌤️ High LCL, Dry Thunderstorm
  ➤ Features     : [190, 2700, 1750, 16, 0.8]
  ➤ Predicted    : No Tornado (confidence: 0.02)
  ➤ Expected     : No Tornado

🌪️ Marginal Shear, High Instability
  ➤ Features     : [240, 3000, 950, 12, 1.6]
  ➤ Predicted    : Tornado (confidence: 0.98)
  ➤ Expected     : Possibly Tornado

🌫️ Cool Humid Blip
  ➤ Features     : [210, 950, 680, 8, 0.6]
  ➤ Predicted    : No Tornado (confidence: 0.02)
  ➤ Expected     : No Tornado

⚡ STP-Peaked Composite Storm
  ➤ Features     : [320, 2800, 870, 20, 3.1]
  ➤ Predicted    : Tornado (confidence: 0.97)
  ➤ Expected     : Tornado

🌀 Low Vorticity High CAPE
  ➤ Features     : [270, 3200, 1000, 10, 0.9]
  ➤ Predicted    : Tornado (confidence: 0.98)
  ➤ Expected     : Edge Case

